In [4]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# Load models and data
with open('../models/good_model.pkl', 'rb') as f:
    good_model = pickle.load(f)
with open('../models/bad_model.pkl', 'rb') as f:
    bad_model = pickle.load(f)
with open('../data/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

X_test = test_data['X_test']
y_test = test_data['y_test']
feature_names = test_data['feature_names']

def get_feature_importances(pipeline_or_model):
    if hasattr(pipeline_or_model, 'named_steps'):
        model = pipeline_or_model.named_steps['classifier']
        return model.feature_importances_
    else:
        return pipeline_or_model.feature_importances_

def test_location_importance(model, feature_names):
    importances = get_feature_importances(model)
    
    location_importance = {
        'neighborhood': sum(importances[6:11]),
        'district': sum(importances[13:22]),
        'rotterdam': sum(importances[i] for i in [11,12])
    }
    
    total_location_importance = sum(location_importance.values())
    total_model_importance = sum(importances)
    
    return {
        'location_importance_ratio': total_location_importance / total_model_importance,
        'location_breakdown': location_importance,
        'interpretation': f"Location features account for {(total_location_importance/total_model_importance)*100:.1f}% of model's decision making"
    }

if __name__ == "__main__":
    print("Testing good model:")
    good_results = test_location_importance(good_model, feature_names)
    print(good_results['interpretation'])
    print("\nLocation breakdown:")
    for area, importance in good_results['location_breakdown'].items():
        print(f"{area}: {importance:.3f}")
    
    print("\nTesting bad model:")
    bad_results = test_location_importance(bad_model, feature_names)
    print(bad_results['interpretation'])
    print("\nLocation breakdown:")
    for area, importance in bad_results['location_breakdown'].items():
        print(f"{area}: {importance:.3f}")

Testing good model:
Location features account for 1.2% of model's decision making

Location breakdown:
neighborhood: 0.002
district: 0.009
rotterdam: 0.001

Testing bad model:
Location features account for 0.9% of model's decision making

Location breakdown:
neighborhood: 0.001
district: 0.007
rotterdam: 0.001
